# Tutorial 4: Tensor Decompositions (CP, Tucker, TT, TR, t-SVD, BTD, HOSVD)

End-to-end overview with runnable, simulated fallbacks when the Tensorus API is unavailable.

In [ ]:
# Lightweight install cell
import sys, subprocess, pkgutil
for p in ['numpy','torch','matplotlib','seaborn','requests']:
    if pkgutil.find_loader(p) is None:
        subprocess.check_call([sys.executable,'-m','pip','install',p])
print('✅ Dependencies ready')

In [ ]:
import time, requests, json, numpy as np, torch
import matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(style='whitegrid')
API='http://127.0.0.1:7860'
def server_ok():
    try:
        return requests.get(f'{API}/health', timeout=2).status_code==200
    except: return False
SERVER=server_ok(); print('📡 Tensorus:', '✅ Connected' if SERVER else '⚠️ Demo Mode')

def post_json(path, payload):
    try:
        r=requests.post(f'{API}{path}', json=payload, timeout=15)
        r.raise_for_status(); return r.json()
    except Exception as e:
        return {'error': str(e), 'demo_mode': True}

## Dataset (3D tensor)

In [ ]:
I,J,K=50,30,12
X = torch.relu(torch.randn(I,J,K)*10+50)
print('Tensor shape:', tuple(X.shape), 'size:', X.numel()*4/1024, 'KB')

## CP Decomposition (rank sweep)

In [ ]:
def cp_simulate(X, rank):
    # Simulate factors and metrics
    factors=[torch.randn(s,rank)*0.1 for s in X.shape]
    comp = X.numel()*4 / (sum(f.numel() for f in factors)*4 + rank*4)
    err = max(0.05, 0.5/rank)
    t = np.random.uniform(0.01,0.05)
    return {'method':'CP','rank':rank,'compression':comp,'error':err,'time_ms':t*1000,'factors':[f.shape for f in factors]}

ranks=[5,10,15,20]
cp_results=[cp_simulate(X,r) for r in ranks]
for r in cp_results: print(r)
plt.figure(); plt.plot(ranks,[r['error'] for r in cp_results],'-o'); plt.title('CP Error vs Rank'); plt.xlabel('rank'); plt.ylabel('relative error'); plt.show()

## Other Methods Overview (Simulated)
We provide light simulations for the remaining methods; replace with Tensorus API calls when available.

In [ ]:
def method_sim(name):
    comp=np.random.uniform(2,20); err=np.random.uniform(0.05,0.25); t=np.random.uniform(8,60)
    return {'method':name,'compression':comp,'error':err,'time_ms':t}
methods=['Tucker','TT','TR','t-SVD','BTD','HOSVD']
summary=[method_sim(m) for m in methods]
for s in summary: print(s)
plt.figure(figsize=(8,3));
plt.subplot(1,2,1); plt.bar([s['method'] for s in summary],[s['compression'] for s in summary]); plt.title('Compression (x)'); plt.xticks(rotation=45)
plt.subplot(1,2,2); plt.bar([s['method'] for s in summary],[s['error'] for s in summary]); plt.title('Relative Error'); plt.xticks(rotation=45); plt.tight_layout(); plt.show()

## When to Use Which
- CP: sparse, interpretable.
- Tucker: balanced compression.
- TT/TR: very high dims / periodic.
- t-SVD: temporal/video.
- BTD: block structure.
- HOSVD: scientific baselines.